This is just a sample code to give you an idea regarding imputation and running an unsupervised model for clustering columns in the helper dataset to get an additional feature to use in the actual dataset. I have modified it so expect some errors. A similar operation is required on the test helper dataset.

In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

In [3]:
tr_h = pd.read_csv('train_helper.csv')
tr = pd.read_csv('train.csv')
df_tr = tr
df_tr_h = tr_h

Imputer begins

- you can use any regressor other than that used here

In [4]:
#imputation for missing data

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

In [5]:
missing_val_cols = ['V20', 'V21']
regressor = RandomForestRegressor()
imputer = IterativeImputer(estimator=regressor, random_state=42)

df_tr_h[missing_val_cols] = imputer.fit_transform(df_tr_h[missing_val_cols])

C:\Users\neela\AppData\Roaming\Python\Python311\site-packages\sklearn\impute\_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Imputer ends

In [14]:
selected_cols = ['V3', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21']

In [12]:
df_tr_h = pd.read_csv('train_helper_NaN.csv')
# df_tr['V5_yearday'] = pd.to_datetime(df_tr['V5']).dt.day_of_year
df_tr_h['V16_month'] = pd.to_datetime(df_tr_h['V16']).dt.month
df_tr_h['V16_day'] = pd.to_datetime(df_tr_h['V16']).dt.day
df_tr_h['V16_hour'] = pd.to_datetime(df_tr_h['V16']).dt.hour
df_tr_h['V16_minute'] = pd.to_datetime(df_tr_h['V16']).dt.minute
try:
    df_tr_h.drop('Unnamed: 0', axis=1, inplace=True)
except:
    pass
unique_cuid = df_tr_h

In [14]:
# unique_cuid = df_tr_h.drop_duplicates(subset=['V1']).reset_index(drop=True)
# unique_eid = df_tr_h.drop_duplicates(subset=['V3']).reset_index(drop=True)
unique_cuid = unique_cuid.groupby('V1')

In [15]:
unique_cuid = unique_cuid.mean()

In [17]:
unique_cuid.drop('Unnamed: 0.1', axis=1, inplace=True)

In [5]:
unique_cuid.drop('Unnamed: 0', axis=1, inplace=True)
# unique_eid.drop('Unnamed: 0', axis=1, inplace=True)

AttributeError: 'DataFrameGroupBy' object has no attribute 'drop'

In [21]:
cuid = pd.DataFrame(unique_cuid['V1'])
# eid = pd.DataFrame(unique_eid['V3'])

KeyError: 'V1'

In [13]:
categorical_cols = ['V3', 'V14', 'V15', 'V16', 'V17']
label_encoder = LabelEncoder()
for col in categorical_cols:
    unique_cuid[col] = label_encoder.fit_transform(unique_cuid[col])
    # unique_eid[col] = label_encoder.fit_transform(unique_eid[col])

Using count encoder

In [121]:
# import category_encoders as ce

# columns_encode = ['V3', 'V14', 'V15', 'V17']
# encoder = ce.CountEncoder(cols=columns_encode)
# unique_cuid = encoder.fit_transform(unique_cuid)

In [18]:
cluster_cols = ['V15', 'V17']#'V16_month', 'V16_day', 'V16_hour', 'V16_minute', 'V17']#, 'V18', 'V19', 'V20', 'V21']
cluster_data_cuid = unique_cuid[cluster_cols]
# cluster_data_eid = unique_eid[cluster_cols]

In [19]:
n_cluster = 3
kmeans_cuid = KMeans(n_clusters=n_cluster, random_state=42)
# kmeans_eid = KMeans(n_clusters=n_cluster, random_state=42)
kmeans_cuid.fit(cluster_data_cuid)
# kmeans_eid.fit(cluster_data_eid)
cluster_labels_cuid = kmeans_cuid.labels_
# cluster_labels_eid = kmeans_eid.labels_

C:\Users\neela\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [22]:
cluster_labels_cuid['V1'] = unique_cuid.index

In [20]:
cluster_labels_cuid = pd.DataFrame(cluster_labels_cuid)
# cluster_labels_eid = pd.DataFrame(cluster_labels_eid)
cluster_labels_cuid['V1'] = cuid['V1']
# cluster_labels_eid['V3'] = eid['V3']

NameError: name 'cuid' is not defined

In [23]:
train = pd.read_csv('train.csv')

In [24]:
train = train.merge(cluster_labels_cuid, on='V1', how='left')
# train = train.merge(cluster_labels_eid, on='V3', how='left')

In [25]:
# train.rename(columns={'0_x': 'column_label_cuid', '0_y': 'coulumn_label_eid'}, inplace=True)
train.rename(columns={0: 'column_label_cuid'}, inplace=True)

get a similar output for the test dataset, remember that the column count should always be the same for train and test

In [26]:
train.to_csv('mod_train.csv')

Creating a supervised model on mod_train dataset (this was a complete waste, couldn't implement much)

In [16]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [4]:
df_tr = pd.read_csv('mod_train.csv')
# df_tr['V5_yearday'] = pd.to_datetime(df_tr['V5']).dt.day_of_year
df_tr['V5_month'] = pd.to_datetime(df_tr['V5']).dt.month
df_tr['V5_day'] = pd.to_datetime(df_tr['V5']).dt.day
# df_tr['V5_hour'] = pd.to_datetime(df_tr['V5']).dt.hour
# df_tr['V13_yearday'] = pd.to_datetime(df_tr['V13']).dt.day_of_year
df_tr['V13_month'] = pd.to_datetime(df_tr['V13']).dt.month
df_tr['V13_day'] = pd.to_datetime(df_tr['V13']).dt.day

In [6]:
df_tr.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
columns_drop = ['V1', 'V2', 'V5', 'V11', 'V13', 'Target']
x_tr = df_tr.drop(columns_drop, axis=1)
y_tr = df_tr['Target']

In [9]:
x_tr

,V3,V4,V6,V7,V8,V9,V10,V12,column_label_cuid,coulumn_label_eid,V5_month,V5_day,V13_month,V13_day
0,EID2062082,485.69445,376474,A,BA1,BB425559,PD44707,CP0,0.0,NaN,3,20,3.0,21.0
1,EID1459012,393.45200,33227,A,BA4793,BB248012,PD39453,CP0,0.0,NaN,4,28,4.0,28.0
2,EID0,375.75000,0,F,BA5631,BB166256,PD237664,CP0,0.0,1.0,8,15,NaN,NaN
3,EID0,6.97225,0,C,BA7188,BB354924,PD99032,CP0,1.0,1.0,6,26,NaN,NaN
4,EID0,16.70000,0,C,BA7236,BB354924,PD63422,CP0,0.0,1.0,9,15,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233628,EID430976,41.75000,391083,A,BA431477,BB282621,PD206909,CP443835,0.0,NaN,9,6,2.0,3.0
233629,EID2012983,83.50000,391083,A,BA431477,BB282621,PD206909,CP443835,0.0,NaN,8,19,2.0,3.0
233630,EID1776806,41.75000,391083,A,BA431477,BB282621,PD206909,CP443835,0.0,NaN,8,26,2.0,3.0
233631,EID631635,167.00000,53,E,BA447678,BB447445,PD0,CP443835,0.0,NaN,4,15,4.0,16.0


In [12]:
categorical_cols = ['V3', 'V7', 'V8', 'V9', 'V10', 'V12']
label_encoder = LabelEncoder()
for col in categorical_cols:
    x_tr[col] = label_encoder.fit_transform(x_tr[col])

In [13]:
x_tr_n, x_val, y_tr_n, y_val = train_test_split(x_tr, y_tr, test_size=0.3, random_state=42)

In [14]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation='relu', input_dim=x_tr_n.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(x_tr_n, y_tr_n, epochs=10, batch_size=32)

Epoch 1/10
5111/5111 [==============================] - 9s 2ms/step - loss: nan - accuracy: 0.9944
Epoch 2/10
5111/5111 [==============================] - 8s 2ms/step - loss: nan - accuracy: 0.9944
Epoch 3/10
5111/5111 [==============================] - 8s 2ms/step - loss: nan - accuracy: 0.9944
Epoch 4/10
5111/5111 [==============================] - 8s 1ms/step - loss: nan - accuracy: 0.9944
Epoch 5/10
5111/5111 [==============================] - 8s 2ms/step - loss: nan - accuracy: 0.9944
Epoch 6/10
5111/5111 [==============================] - 8s 2ms/step - loss: nan - accuracy: 0.9944
Epoch 7/10
5111/5111 [==============================] - 8s 2ms/step - loss: nan - accuracy: 0.9944
Epoch 8/10
5111/5111 [==============================] - 9s 2ms/step - loss: nan - accuracy: 0.9944
Epoch 9/10
5111/5111 [==============================] - 9s 2ms/step - loss: nan - accuracy: 0.9944
Epoch 10/10
5111/5111 [==============================] - 9s 2ms/step - loss: nan - accuracy: 0.9944


In [29]:
test_df = pd.read_csv('mod_test.csv')
test_df['Target'] = np.random
# df_tr['V5_yearday'] = pd.to_datetime(df_tr['V5']).dt.day_of_year
test_df['V5_month'] = pd.to_datetime(test_df['V5'], dayfirst=1).dt.month
test_df['V5_day'] = pd.to_datetime(test_df['V5'], dayfirst=1).dt.day
# test_df['V5_hour'] = pd.to_datetime(test_df['V5'], dayfirst=1).dt.hour
# df_tr['V13_yearday'] = pd.to_datetime(df_tr['V13']).dt.day_of_year
test_df['V13_month'] = pd.to_datetime(test_df['V13'], dayfirst=1).dt.month
test_df['V13_day'] = pd.to_datetime(test_df['V13'], dayfirst=1).dt.day

In [30]:
x_test = test_df.drop(columns_drop, axis=1)

In [31]:
for col in categorical_cols:
    x_test[col] = label_encoder.fit_transform(x_test[col])

In [34]:
x_test.drop('Unnamed: 0', axis=1, inplace=True)

In [35]:
prediction = model.predict(x_test)

6746/6746 [==============================] - 8s 1ms/step


In [36]:
probab = prediction[:,0]

In [37]:
a = prediction[prediction>0]

In [38]:
len(a)

0